0. Upload your dataset in Google Drive and mount in Colab. (If you want to train your own dataset)
1. Check if the selected GPU has CUDA.

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
    print("CUDA version:", torch.version.cuda)
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")


2. Clone Gaussian Splatting repository.

In [ ]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

3. Fetch sample data. Optional if you have your own dataset and mounted it via Drive. Change the name of /images folder to /input. Ex: .../db/playroom/images => .../db/playroom/input

In [ ]:
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

4. Install dependencies for COLMAP:

In [ ]:
!sudo apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-regex-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev \
    libsuitesparse-dev \
    libgflags-dev \
    libpng-dev \
    libjpeg-dev \
    libtiff-dev \
    libxxf86vm1 \
    libxxf86vm-dev \
    libxi-dev \
    libxrandr-dev

5. Build COLMAP. Takes ~30 minutes.

In [ ]:
%cd /content
!git clone https://github.com/colmap/colmap.git
%cd colmap
!mkdir build
%cd build
!cmake ..
!make -j$(nproc)
!sudo make install

6. Convert the dataset using COLMAP. <br />
Change the path to your own dataset or use the sample if you just want to try things out.

In [ ]:
%cd /content/gaussian-splatting
!python convert.py -s {path_to_dataset} #/content/gaussian-splatting/db/playroom

7. Train the converted dataset.

In [ ]:
# Replace {path_to_dataset}
# --save_iterations: arg for saved checkpoints. Set different number if you want.
!python train.py -s {path_to_dataset} --iterations 30000 --save_iterations 10000 20000 30000   #/content/gaussian-splatting/db/playroom

8. Download the .ply file in .../output/{sha}/point_cloud/{iteration_num}/point_cloud.py